In [1]:
from dotenv import load_dotenv
from google import genai

load_dotenv()
gemini_client = genai.Client()

In [2]:
def list_available_embedding_models():
    for model in gemini_client.models.list():
        if model.supported_actions is None:
            continue
        if "embedContent" in model.supported_actions:
            print(model.name)

list_available_embedding_models()

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


In [3]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
    def __init__(self):
        pass

    def __call__(self, input: Documents) -> Embeddings:
        EMBEDDING_MODEL = "gemini-embedding-001"
        response = gemini_client.models.embed_content(
            model=EMBEDDING_MODEL,
            contents=input,
            config=types.EmbedContentConfig(
                task_type="SEMANTIC_SIMILARITY",
                output_dimensionality=3072,
            ),
        )
        if response.embeddings is None:
            raise RuntimeError("No embeddings returned")
        if response.embeddings[0].values is None:
            raise RuntimeError("No embedding values returned")
        return response.embeddings[0].values  # type: ignore

gemini_embedding_function = GeminiEmbeddingFunction()

In [4]:
import chromadb
from chromadb.errors import NotFoundError

chroma_client = chromadb.PersistentClient()
try:
    db = chroma_client.get_collection(
        name="rag-db", embedding_function=gemini_embedding_function
    )
except NotFoundError:
    db = chroma_client.create_collection(
        name="rag-db", embedding_function=gemini_embedding_function
    )

In [5]:
def embed_document(text, db):
    parts = []
    current_part = ""
    for c in text:
        current_part += c
        if len(current_part) >= 500 or c == "\n":
            current_part = current_part.strip()
            if current_part == "" or current_part.startswith("#"):
                current_part = ""
                continue
            parts.append(current_part.strip())
            current_part = ""

    for i, part in enumerate(parts):
        db.add(documents=[part], ids=[f"doc-{i}"])

with open("document.md", "r", encoding="utf-8") as f:
    document = f.read()

embed_document(document, db)

In [20]:
def get_relevant_passages(query, db):
    passage = db.query(query_texts=[query], n_results=3)["documents"][0]
    return passage

In [7]:
def make_prompt(query, passage):
    prompt="""
        You are a helpful and informative bot that answers questions using text from the reference passage included below.
        Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
        If the passage is irrelevant to the answer, you may ignore it.

        QUESTION: '{query}'

        PASSAGE: '{passages}'
        
        ANSWER:
    """
    return prompt.format(
        query=query,
        passages=passage
    )

In [8]:
def generate_answer(query, passage):
    CHAT_MODEL = "gemini-2.5-flash-lite"
    response = gemini_client.models.generate_content(
        model=CHAT_MODEL,
        contents=make_prompt(query, passage),
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0),
        )
    )
    return (
        response.usage_metadata.total_token_count if response.usage_metadata else None,
        response.text,
    )

In [9]:
def test_with_rag(query, db):
    passages = get_relevant_passages(query, db)
    answer = generate_answer(query, "".join(passages))
    return answer

def test_with_context(query, long_context):
    answer = generate_answer(query, long_context)
    return answer

## Testcases

In [21]:
query = "神經織網技術的發明者是誰？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (530, '神經織網（Neuro-Loom）系統的發明者是 Evelyn Reed 博士。')
Context: (2699, '神經織網技術的發明者是天才工程師 Evelyn Reed 博士，她於 2011 年發明了「神經織網」（Neuro-Loom）系統，該系統是一個高度複雜的腦機介面，利用量子點作為神經標記物並結合高解析度的功能性磁振造影（fMRI）即時追蹤技術，能夠以前所未有的精度即時監測並繪製出大腦在處理特定資訊時的活躍路徑。')


In [22]:
query = "旨在規範織夢技術、於 2042 年頒布的法案叫什麼名字？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (500, '旨在規範腦機介面與意識干預技術、於 2042 年頒布的法案叫做《關於規範腦機介面與意識干預技術的倫理準則》，史稱《墨菲斯法案》（The "Morpheus" Act）。')
Context: (2634, '旨在規範織夢技術、於 2042 年頒布的法案稱為《墨菲斯法案》（The "Morpheus" Act）。')


In [23]:
query = "Aris Thorne 博士最初發起的專案計畫名稱是什麼？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (647, 'Aris Thorne 博士最初發起的專案計畫名稱是「Lucid」。')
Context: (2615, 'Aris Thorne 博士於 1985 年發起的專案計畫名稱是「Lucid」。')


In [24]:
query = "織夢技術的核心技術原理包含哪三個主要部分？請簡要說明它們各自的作用。"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (375, '織夢技術的三大核心技術原理分別是：記憶印痕存取 (Memory Engram Access, MEA)，它能夠利用「神經織網」系統來定位並讀取與特定記憶相關的「印痕細胞」集群，同時也能夠將新的資訊模式寫入大腦形成虛構的記憶。')
Context: (2711, '織夢技術的核心技術原理包含三個主要部分：睡夢波誘導（Somno-Wave Induction, SWI），用於引導使用者大腦進入最適合織夢的特定過渡狀態；記憶印痕存取（Memory Engram Access, MEA），用於定位、讀取與特定記憶相關的「印痕細胞」集群，並能寫入新的資訊模式；以及感官回饋模擬（Sensory Feedback Simulation, SFS），用於直接刺激大腦的感覺皮層，生成逼真的五感體驗。')


In [25]:
query = "為什麼織夢技術能夠應用於創傷後壓力症候群（PTSD）的治療？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (572, '織夢技術能夠應用於創傷後壓力症候群（PTSD）的治療，是因為患者可以在一個絕對安全的夢境環境中，在治療師的引導下，以一種可控的方式反覆面對和處理他們的創傷記憶，例如，經歷過嚴重車禍的患者可以在夢中逐步接近事故現場，從而降低創傷事件帶來的恐懼反應，大大縮短療程。')
Context: (2706, '織夢技術之所以能夠應用於創傷後壓力症候群（PTSD）的治療，是因為它能在一個絕對安全的夢境環境中，讓患者在治療師的引導下，以一種可控的方式反覆面對和處理他們的創傷記憶，例如，經歷過嚴重車禍的患者可以在夢中逐步從遠處觀察事故現場，最終以旁觀者的身份完整經歷事件，從而逐步降低創傷事件帶來的恐懼反應，大大縮短了療程。')


In [26]:
query = "請總結織夢技術在「加速學習」和「專業培訓」這兩個領域的應用場景。"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (330, '織夢技術在「加速學習」領域的應用場景包括語言學習者在夢中與虛擬母語人士進行數小時的流利對話，藉由大腦在睡眠中自動強化語言迴路，以及醫學生在虛擬手術台上反覆練習複雜手術流程，直到形成肌肉記憶，這能將數週或數月的刻意練習濃縮在幾晚的睡眠之中，顯著提升學習效率。')
Context: (2751, '織夢技術在「加速學習」領域的應用，體現在語言學習者可以在夢中與虛擬的母語人士進行數小時的流利對話，從而加強相關語言迴路，以及醫學生可以在虛擬手術台上反覆練習複雜的手術流程，以形成肌肉記憶，實現學習效率的指數級提升；而在「專業培訓」領域，該技術則被用於航空公司模擬極端的飛行故障，讓飛行員在高壓環境下進行決策訓練，以及消防部門演練火場救援，這些訓練都能在零風險的條件下，並包含壓力、高溫、體力消耗等生理感受的模擬，效果遠超傳統模擬器。')


In [27]:
query = "「神經織網」系統相較於早期的「Lucid」計畫，最關鍵的技術進步是什麼？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (846, '「神經織網」系統相較於早期的「Lucid」計畫，最關鍵的技術進步在於它不僅能「讀取」夢境線索，還能透過精準的聚焦超音波脈衝對其進行「寫入」操作，進而逆向活化特定的神經通路，使研究人員首次擁有了建構夢境內容的能力，例如在 2020 年成功為受試者植入了一段包含視覺、聽覺和觸覺的簡單夢境片段，而「Lucid」計畫僅能誘導「清醒夢」，讓做夢者意識到自己在做夢，卻無法對夢境內容進行實質干預或塑造。')
Context: (2683, '相較於早期的「Lucid」計畫，能夠讀取並寫入夢境線索、從而建構可預設夢境的「神經織網」系統是一項關鍵技術進步，因為「Lucid」計畫僅能誘導清醒夢，讓做夢者意識到自己在做夢，卻無法干預或塑造夢境內容。')


In [28]:
query = "為什麼《墨菲斯法案》的頒布是必要的？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (579, '《墨菲斯法案》的頒布是必要的，因為它旨在應對腦機介面與意識干預技術帶來的挑戰，特別是針對織夢技術的商業應用，並確立了「思想與記憶的絕對隱私權」，以防止惡意行為者植入虛假記憶或抹去關鍵記憶，從而顛覆司法系統和個人身份認同。')
Context: (2657, '《墨菲斯法案》的頒布是必要的，旨在應對織夢技術帶來的倫理困境，例如記憶篡改與偽造、夢境隱私的侵犯以及潛在的心理依賴等問題，並由此確立了「思想與記憶的絕對隱私權」。')


In [32]:
query = "文件中是否提到了織夢技術在軍事領域的任何應用？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (456, '根據提供的文本，文件中並未提及織夢技術在軍事領域的任何應用，而是重點闡述了「共享夢境」技術在遠距協作、家庭團聚及社交模式上的預期應用，以及與腦機介面整合後在意識數位化和延伸方面的想像空間，同時也探討了這項技術成熟後所帶來的倫理困境和哲學爭議。')
Context: (2611, '根據提供的文件，並未提及織夢技術在軍事領域的任何應用。')


In [30]:
query = "Aris Thorne 博士是否直接參與了「神經織網」的發明？"
print("RAG:", test_with_rag(query, db))
print("Context:", test_with_context(query, document))

RAG: (583, 'Aris Thorne 博士並未直接參與「神經織網」（Neuro-Loom）系統的發明，而是他的學生 Evelyn Reed 博士基於 Thorne 博士關於夢境是可探索且可被引導的內在世界的理念，於 2011 年發明了該系統。')
Context: (2755, 'Aris Thorne 博士雖然是織夢技術的奠基者，他在 1980 年代末期發起了「Lucid」專案，旨在開發能夠穩定誘發「清醒夢」的設備，但「神經織網」（Neuro-Loom）系統是由他的學生 Evelyn Reed 博士在 21 世紀初發明的，這項技術能夠從內部進行精密介接，透過繪製神經通路圖譜來編織夢境，這才達到了真正的「織夢」能力，而 Thorne 博士最初的「Lucid」計畫僅僅是提供了一把「鑰匙」，讓做夢者能夠「意識到」自己在做夢，但無法干預或塑造夢境內容。')
